In [1]:
from Dataload import dataload

from module import transformer,video_model,invres

import torch
import torch.nn as nn
from torchsummary import summary
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
from torchvision import  utils
import os
from constant import EMOTIPATH
import time
from tqdm.notebook import tqdm


In [2]:
#Batchfy
#Counts number of correct label
def num_correct(prediction,labels):
    correct=0
    for i,(pred_label,label) in enumerate(zip(prediction,labels)):
        if (pred_label.item()==label.item()):
            correct +=1
    return correct

Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video=os.path.join(EMOTIPATH,"Train")
Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_video=os.path.join(EMOTIPATH,"Val")

frame_num=5
train_data=dataload.Video_Frame_Data(Train_label,"train.csv",base_path_v=Train_video,frame_num=frame_num)
valid_data=dataload.Video_Frame_Data(Val_labels,"valid.csv",base_path_v=Val_video,frame_num=frame_num)

#train_data1=dataload.Video_Embedding_Data("train.csv",Train_label)
#valid_data=dataload.Video_Embedding_Data("valid.csv",Val_labels)
            
train_dataloader = DataLoader(train_data, batch_size=32
                       , num_workers=0,shuffle=True)
#valid_data=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video,strict_num=26)

valid_dataloader = DataLoader(valid_data, batch_size=32
                       , num_workers=0)



len(train_data)

2661

In [3]:
valid_data[60][0].dtype

torch.float64

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)
model=video_model.Video_Feature(joint=False,grayscale=False,classfication=True,frame=frame_num)
for name, child in model.named_children():
   if not name in ['densenet']:
       print(name + ' is unfrozen')
       for param in child.parameters():
           param.requires_grad = True
   else:
       print(name + ' is frozen')
       for param in child.parameters():
           param.requires_grad = False
#model=model.to(device)  
model=model.double().to(device)
#model=model.float()
criterion = nn.CrossEntropyLoss()  
loss_func = torch.nn.MSELoss()

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0002, betas=(0.5, 0.999))


densenet is frozen
fc2 is unfrozen
fc3 is unfrozen
posencoding is unfrozen
transformer is unfrozen


In [6]:
def train(num_epochs,name,model):
    f = open(name+".txt",'a')
    start = time.time()
    #Triaining
    train_loss=[]
    valid_accuracy=[]
    model.train()
    for epochs in range(0,num_epochs):
        first=True
        model.train()
        correct=0
        total_samples=0
        avg_tloss=0
        print("Training Epoch: ", epochs+1,"\n")
        for i_batch, (sample_batched,_,label) in tqdm(enumerate(train_dataloader)):
           # sample_batched=sample_batched.frames
            batch_size=sample_batched.size(0)
            optimizer.zero_grad()
            sample_batched=sample_batched.to(device)
            output=model(sample_batched)
            loss=criterion(output,label.to(device))
            loss.backward()
            predicted = torch.max(output, 1)
            prediction=predicted.indices.detach().cpu()
            correct +=num_correct(prediction,label)
            total_samples+=batch_size
            accuracy=correct/(total_samples)
            optimizer.step()
            true_label=label.detach().cpu()
            avg_tloss+=loss.item()
            if first:
                first=False
                conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
            else:
                conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
            if i_batch%20==0:
               # print(label)
                print("Batch: ",i_batch+1,"/",len(train_dataloader))
                print("Batch Recognition loss: ", loss.item())

        print(conf_mat)
        avg_tloss=avg_tloss/len(train_dataloader)
        avg_taccuracy=correct/total_samples
        print("Average_Loss: ",avg_tloss)
        print("Average_Accuracy: ",avg_taccuracy)

        torch.save(model.state_dict(),name)
        print("Validation\n")
        model.eval()   
        correct=0
        total_samples=0
        avg_vloss=0
        first=True

        for i_batch, (sample_batched,_,label) in enumerate(valid_dataloader):
            batch_size=sample_batched.size(0)
            sample_batched=sample_batched.to(device)
            output=model(sample_batched)
            loss=criterion(output,label.to(device))
            avg_vloss+=loss.item()
            predicted = torch.max(output, 1)
            prediction=predicted.indices.detach().cpu()
            correct +=num_correct(prediction,label)
            total_samples+=batch_size

            true_label=label.detach().cpu()
            if first:
                first=False
                conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
            else:
                conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
        print(conf_mat)
        avg_vloss=avg_vloss/len(valid_dataloader)
        print(avg_vloss)
        avg_vaccuracy=correct/(total_samples)
        print("Accuracy: ", avg_vaccuracy)

       # data = " %d,%d,%d,%d \n" % (avg_tloss,avg_taccuracy,avg_vloss,avg_vaccuracy)



    f.close()


In [7]:
model_name="pre_embedded_5_frames"
train(20,model_name,model)

Training Epoch:  1 



Batch:  1 / 84
Batch Recognition loss:  1.09314688594941
Batch:  21 / 84
Batch Recognition loss:  1.0191039765275929
Batch:  41 / 84
Batch Recognition loss:  1.0166374573876193
Batch:  61 / 84
Batch Recognition loss:  1.0508648426365186
Batch:  81 / 84
Batch Recognition loss:  1.0672927621313324

[[522   0 284]
 [604   0 319]
 [285   0 647]]
Average_Loss:  1.048527383433934
Average_Accuracy:  0.4393085306275836
Validation

[[189   0 110]
 [214   0  67]
 [ 38   0 148]]
1.0846347214347585
Accuracy:  0.43994778067885115
Training Epoch:  2 



Batch:  1 / 84
Batch Recognition loss:  1.0000864147496322
Batch:  21 / 84
Batch Recognition loss:  1.0209347529964132
Batch:  41 / 84
Batch Recognition loss:  0.9015418986115982
Batch:  61 / 84
Batch Recognition loss:  1.0098727011112965
Batch:  81 / 84
Batch Recognition loss:  0.9832591385183491

[[589   1 216]
 [675   0 248]
 [238   0 694]]
Average_Loss:  1.008436632755505
Average_Accuracy:  0.48214956783164226
Validation

[[245   0  54]
 [234   0  47]
 [ 61   0 125]]
1.018902190132702
Accuracy:  0.4830287206266319
Training Epoch:  3 



Batch:  1 / 84
Batch Recognition loss:  1.047266420082255
Batch:  21 / 84
Batch Recognition loss:  0.9388785280071814
Batch:  41 / 84
Batch Recognition loss:  0.9548855153158153
Batch:  61 / 84
Batch Recognition loss:  0.9996705107195986
Batch:  81 / 84
Batch Recognition loss:  0.9285935915907997

[[652   0 154]
 [692   0 231]
 [223   0 709]]
Average_Loss:  0.9746558872055214
Average_Accuracy:  0.5114618564449455
Validation

[[247   0  52]
 [237   0  44]
 [101   0  85]]
1.058910591147364
Accuracy:  0.43342036553524804
Training Epoch:  4 



Batch:  1 / 84
Batch Recognition loss:  1.026489743145249
Batch:  21 / 84
Batch Recognition loss:  0.8469583821396199
Batch:  41 / 84
Batch Recognition loss:  1.0274250328917895
Batch:  61 / 84
Batch Recognition loss:  1.0595961593510763
Batch:  81 / 84
Batch Recognition loss:  0.9140614708289088

[[660   0 146]
 [693   0 230]
 [215   0 717]]
Average_Loss:  0.9639896135000893
Average_Accuracy:  0.5174746335963923
Validation

[[207   0  92]
 [208   0  73]
 [ 45   0 141]]
1.0474229801325727
Accuracy:  0.45430809399477806
Training Epoch:  5 



...

Batch:  1 / 84
Batch Recognition loss:  0.7896489650483478
Batch:  21 / 84
Batch Recognition loss:  0.8315218353133731
Batch:  41 / 84
Batch Recognition loss:  0.829845972246387
Batch:  61 / 84
Batch Recognition loss:  0.903336709874529
Batch:  81 / 84
Batch Recognition loss:  0.9473607848934534

[[736   0  70]
 [769   0 154]
 [157   0 775]]
Average_Loss:  0.8912996514776723
Average_Accuracy:  0.5678316422397595
Validation

[[202   0  97]
 [216   0  65]
 [ 38   0 148]]
1.044528019870427
Accuracy:  0.45691906005221933
Training Epoch:  17 



Batch:  1 / 84
Batch Recognition loss:  0.8740529587545992
Batch:  21 / 84
Batch Recognition loss:  0.8642028071888389
Batch:  41 / 84
Batch Recognition loss:  0.8787183792430536
Batch:  61 / 84
Batch Recognition loss:  0.8117245431285859
Batch:  81 / 84
Batch Recognition loss:  0.907729001485513

[[729   0  77]
 [767   0 156]
 [148   0 784]]
Average_Loss:  0.8868151376683278
Average_Accuracy:  0.5685832393836904
Validation

[[268   0  31]
 [245   0  36]
 [108   0  78]]
1.0280959923742787
Accuracy:  0.4516971279373368
Training Epoch:  18 



Batch:  1 / 84
Batch Recognition loss:  0.8677951430144694
Batch:  21 / 84
Batch Recognition loss:  0.9526891079699809
Batch:  41 / 84
Batch Recognition loss:  0.8627462554980646
Batch:  61 / 84
Batch Recognition loss:  0.8919870511363921
Batch:  81 / 84
Batch Recognition loss:  0.8967352408009495

[[739   0  67]
 [747   0 176]
 [150   0 782]]
Average_Loss:  0.8843209835608251
Average_Accuracy:  0.5715896279594137
Validation

[[286   0  13]
 [267   0  14]
 [165   0  21]]
1.0962828389083992
Accuracy:  0.4007832898172324
Training Epoch:  19 



Batch:  1 / 84
Batch Recognition loss:  0.8959200067212423
Batch:  21 / 84
Batch Recognition loss:  0.8515954335167202
Batch:  41 / 84
Batch Recognition loss:  0.8779243493614618
Batch:  61 / 84
Batch Recognition loss:  0.886039388651647
Batch:  81 / 84
Batch Recognition loss:  0.9125868570708777

[[711   0  95]
 [748   0 175]
 [229   0 703]]
Average_Loss:  0.9355518378775081
Average_Accuracy:  0.5313791807591132
Validation

[[233   0  66]
 [228   0  53]
 [101   0  85]]
1.0609189060803834
Accuracy:  0.4151436031331593
Training Epoch:  20 



Batch:  1 / 84
Batch Recognition loss:  0.9936416735892304
Batch:  21 / 84
Batch Recognition loss:  0.909139766019582
Batch:  41 / 84
Batch Recognition loss:  1.0144843973342754
Batch:  61 / 84
Batch Recognition loss:  0.8928965056549255
Batch:  81 / 84
Batch Recognition loss:  0.9447122266107026

[[719   0  87]
 [742   0 181]
 [208   0 724]]
Average_Loss:  0.9232164562231922
Average_Accuracy:  0.5422773393461104
Validation

[[242   0  57]
 [233   0  48]
 [ 63   0 123]]
1.0407181983358358
Accuracy:  0.4765013054830287


In [ ]:
    model.eval()   
    correct=0
    total_samples=0
    avg_vloss=0
    first=True
    for i_batch, (sample_batched,_,label) in enumerate(valid_dataloader):
        batch_size=sample_batched.size(0)
        sample_batched=sample_batched.to(device)
        output=model(sample_batched)
        loss=criterion(output,label.to(device))
      #  loss=loss_func(output,label.float().to(device))
        avg_vloss+=loss.item()
        predicted = torch.max(output, 1)
        prediction=predicted.indices.detach().cpu()
        correct +=num_correct(prediction,label)
        total_samples+=batch_size
        
        true_label=label.detach().cpu()
        if first:
            first=False
            conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
        else:
            conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
    print(conf_mat)
    avg_vloss=avg_vloss/len(valid_dataloader)
    print(avg_vloss)
    avg_vaccuracy=correct/(total_samples)
    print("Accuracy: ", avg_vaccuracy)
    
    data = " %d,%d,%d,%d \n" % (avg_tloss,avg_taccuracy,avg_vloss,avg_vaccuracy)


In [ ]:
valid_data1=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video,strict_num=26)

valid_dataloader1 = DataLoader(valid_data1, batch_size=32
                       , num_workers=0)

In [ ]:
    model.eval()   
    correct=0
    total_samples=0
    avg_vloss=0
    first=True
    model=model.float()
    for i_batch, (sample_batched,_,label) in enumerate(valid_dataloader1):
        batch_size=sample_batched.size(0)
        sample_batched=sample_batched.to(device)
        sample_batched=model.posencoding(model.stack_frame(sample_batched))
        output=model(sample_batched)
        loss=criterion(output,label.to(device))
      #  loss=loss_func(output,label.float().to(device))
        avg_vloss+=loss.item()
        predicted = torch.max(output, 1)
        prediction=predicted.indices.detach().cpu()
        correct +=num_correct(prediction,label)
        total_samples+=batch_size
        
        true_label=label.detach().cpu()
        if first:
            first=False
            conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
        else:
            conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
    print(conf_mat)
    avg_vloss=avg_vloss/len(valid_dataloader)
    print(avg_vloss)
    avg_vaccuracy=correct/(total_samples)
    print("Accuracy: ", avg_vaccuracy)
    
   # data = " %d,%d,%d,%d \n" % (avg_tloss,avg_taccuracy,avg_vloss,avg_vaccuracy)


In [ ]:
sample_batched

In [ ]:
for i_batch, (sample_batched,label) in enumerate(valid_dataloader):
    debug=sample_batched
    break

In [ ]:
debug

In [ ]:

        output=model(sample_batched)
                                         
        loss=criterion(output,label.to(device))
      #  loss=loss_func(output,label.float().to(device))
        avg_vloss+=loss.item()
        predicted = torch.max(output, 1)
        prediction=predicted.indices.detach().cpu()
        correct +=num_correct(prediction,label)
        total_samples+=batch_size
        
        true_label=label.detach().cpu()
        if first:
            first=False
            conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
        else:
            conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
    print(conf_mat)
    avg_vloss=avg_vloss/len(valid_dataloader)
    print(avg_vloss)
    avg_vaccuracy=correct/(total_samples)
    print("Accuracy: ", avg_vaccuracy)

In [ ]:
train(1)